In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from astropy.table import Table

In [ ]:
input1 = Table()
input1['ID'] = [f'CL{i}' for i in range(5)]
input1['RA'] = [0., .0001, 10, 25, 20]
input1['DEC'] = 0.
input1['Z'] = [.2, .3, .25, .4, .35]

input2 = input1[:4]
input2['Z'][:2] = [.3, .2]

In [ ]:
import sys
sys.path.append('../')

In [ ]:
from clevar.catalog import Catalog

In [ ]:
c1 = Catalog(id=input1['ID'], ra=input1['RA'], dec=input1['DEC'], z=input1['Z'])
c2 = Catalog(id=input2['ID'], ra=input2['RA'], dec=input2['DEC'], z=input2['Z'])
display(c1.data)
display(c2.data)

In [ ]:
c1._init_match_vals()
c2._init_match_vals()
display(c1.match)
display(c2.match)

In [ ]:
from clevar.match.proximity import ProximityMatch

In [ ]:
mt = ProximityMatch()

In [ ]:
mt_config = {'delta_z':.2,
            'match_radius': '1 arcsec'}
mt._prep_for_match(c1, config=mt_config)
mt._prep_for_match(c2, config=mt_config)

In [ ]:
display(c1.mt_input)
display(c2.mt_input)

In [ ]:
mt_config = {'delta_z':.2,
            'match_radius': '1 degree'}
mt.multiple(c1, c2)

In [ ]:
mt.multiple(c2, c1)

In [ ]:
mt.unique(c1, c2, 'ang')
mt.unique(c2, c1, 'ang')

In [ ]:
display(c1.match)
display(c2.match)

In [ ]:
c1.cross_match()
c2.cross_match()

In [ ]:
display(c1.match)
display(c2.match)

In [ ]:
mt.save_matches(c1, c2, {'out_dir':'temp'})

In [ ]:
temp = Table.read('temp/match1.fits')

In [ ]:
temp

In [ ]:
mt.load_matches(c1, c2, {'out_dir':'temp'})
display(c1.match)
display(c2.match)